In [12]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import random

In [13]:
import tkinter as tk
import pandas as pd
import random

class MovieRecommendationSystem:
    def __init__(self, csv_file):
        self.movies = pd.read_csv(csv_file).to_dict(orient='records')
        self.genre_weights = {
            'Sci-Fi': 7,
            'Action': 6,
            'Drama': 5,
            'Comedy': 4,
            'Horror': 3,
            'Romance': 2,
            'Animation': 2,
            'Thriller': 4
        }
        self.cast_weights = {
            'A-list': 5,
            'B-list': 3,
            'C-list': 1
        }
        self.production_weights = {
            'Warner Bros': 5,
            'Disney': 4.5,
            '20th Century Fox': 4.5,
            'Bhansali': 4,
            'AA Films': 3.5,
            'T-series': 3,
            'Aamir Khan Productions': 4,
            'Arka Media Works': 4,
            'Marvel Studios': 5,
            'Barunson E&A': 3.5,
            'Vinod Chopra Films': 3.5,
            'Sony Pictures Networks': 3,
            'Zee Studios': 3,
            'GP Sippy': 3,
            'Phantom Films': 3,
            'Cine1 Studios': 2.5,
            'Hombale Films': 3,
            'Cape of Good Films': 3,
            'Sohum Shah Films': 2,
            'Maddock Films': 3,
            'Matchbox Pictures': 3,
            'Yash Raj Films': 4,
            'Salman Khan Films': 3.5,
            'Other': 1
        }
        self.budget_weights = {
            'High': 5,
            'Medium': 3,
            'Low': 1
        }
        self.box_office_weights = {
            'Blockbuster': 5,
            'Hit': 4,
            'Average': 3,
            'Flop': 1
        }

    def add_movie(self, title, genre, cast_quality, production, budget, box_office, critical_acclaim):
        rating = self.calculate_rating(genre, cast_quality, production, budget, box_office, critical_acclaim)
        new_movie = {
            'title': title,
            'genre': genre,
            'cast_quality': cast_quality,
            'production': production,
            'budget': budget,
            'box_office': box_office,
            'critical_acclaim': critical_acclaim,
            'rating': rating
        }
        self.movies.append(new_movie)
        self.ensure_unique_ratings()

    def calculate_rating(self, genre, cast_quality, production, budget, box_office, critical_acclaim):
        """Calculate rating based on multiple factors."""
        genre_weight = self.genre_weights.get(genre, 0)
        cast_weight = self.cast_weights.get(cast_quality, 0)
        production_weight = self.production_weights.get(production, 0)
        budget_weight = self.budget_weights.get(budget, 0)
        box_office_weight = self.box_office_weights.get(box_office, 0)
        
        base_score = genre_weight + cast_weight + production_weight + budget_weight + box_office_weight + (critical_acclaim / 2)
        
        # Normalize to 10
        max_score = 30  # Adjust max_score based on the sum of maximum weights
        normalized_score = (base_score / max_score) * 10
        
        return normalized_score

    def ensure_unique_ratings(self):
        """Ensure all ratings are unique by slightly adjusting them."""
        seen_ratings = {}
        for movie in self.movies:
            rating = movie['rating']
            while rating in seen_ratings:
                rating += random.uniform(-0.1, 0.1)
            seen_ratings[rating] = True
            movie['rating'] = rating

    def search_movies(self, query, search_by='title'):
        self.movies = self.quicksort(self.movies, key=search_by)  # Sort movies by the search key before searching
        return self.binary_search(query, search_by)

    def quicksort(self, movies, key='rating'):
        if len(movies) <= 1:
            return movies
        pivot = movies[len(movies) // 2]
        left = [x for x in movies if x[key] < pivot[key]]
        middle = [x for x in movies if x[key] == pivot[key]]
        right = [x for x in movies if x[key] > pivot[key]]
        return self.quicksort(left, key) + middle + self.quicksort(right, key)

    def binary_search(self, query, key='title'):
        low, high = 0, len(self.movies) - 1
        results = []
        while low <= high:
            mid = (low + high) // 2
            if self.movies[mid][key].lower() == query.lower():
                results.append(self.movies[mid])
                # Check adjacent elements for multiple results
                i = mid - 1
                while i >= 0 and self.movies[i][key].lower() == query.lower():
                    results.append(self.movies[i])
                    i -= 1
                i = mid + 1
                while i < len(self.movies) and self.movies[i][key].lower() == query.lower():
                    results.append(self.movies[i])
                    i += 1
                break
            elif self.movies[mid][key].lower() < query.lower():
                low = mid + 1
            else:
                high = mid - 1
        return results

    def recommend_movies(self, movies, top_n):
        sorted_movies = self.quicksort(movies)
        return sorted_movies[:top_n]

    def delete_movie(self, title):
        self.movies = [movie for movie in self.movies if movie['title'].lower() != title.lower()]

    def save_movies(self, csv_file):
        df = pd.DataFrame(self.movies)
        df.to_csv(csv_file, index=False)


class MovieApp:
    def __init__(self, root, system):
        self.system = system
        self.root = root
        self.root.title("Prnsh's CineMatch - MovieRecommendationSystem")

        self.create_widgets()

    def create_widgets(self):
        # Configure grid weight for root
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_columnconfigure(1, weight=1)
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_rowconfigure(1, weight=1)
        self.root.grid_rowconfigure(2, weight=1)

        # Add Movie Frame
        self.add_frame = tk.Frame(self.root, bd=2, relief=tk.SUNKEN)
        self.add_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

        tk.Label(self.add_frame, text="Add Movie").grid(row=0, column=0, columnspan=2)

        tk.Label(self.add_frame, text="Title").grid(row=1, column=0)
        self.title_entry = tk.Entry(self.add_frame)
        self.title_entry.grid(row=1, column=1)

        tk.Label(self.add_frame, text="Genre").grid(row=2, column=0)
        self.genre_entry = tk.Entry(self.add_frame)
        self.genre_entry.grid(row=2, column=1)

        tk.Label(self.add_frame, text="Cast Quality").grid(row=3, column=0)
        self.cast_entry = tk.Entry(self.add_frame)
        self.cast_entry.grid(row=3, column=1)

        tk.Label(self.add_frame, text="Production").grid(row=4, column=0)
        self.production_entry = tk.Entry(self.add_frame)
        self.production_entry.grid(row=4, column=1)

        tk.Label(self.add_frame, text="Budget").grid(row=5, column=0)
        self.budget_entry = tk.Entry(self.add_frame)
        self.budget_entry.grid(row=5, column=1)

        tk.Label(self.add_frame, text="Box Office").grid(row=6, column=0)
        self.box_office_entry = tk.Entry(self.add_frame)
        self.box_office_entry.grid(row=6, column=1)

        tk.Label(self.add_frame, text="Critical Acclaim").grid(row=7, column=0)
        self.critical_entry = tk.Entry(self.add_frame)
        self.critical_entry.grid(row=7, column=1)

        self.add_button = tk.Button(self.add_frame, text="Add Movie", command=self.add_movie, bg='blue', fg='white')
        self.add_button.grid(row=8, column=0, columnspan=2, pady=5)

        # Search Movie Frame
        self.search_frame = tk.Frame(self.root, bd=2, relief=tk.SUNKEN)
        self.search_frame.grid(row=0, column=1, padx=10, pady=10, sticky="nsew")

        tk.Label(self.search_frame, text="Search Movie").grid(row=0, column=0, columnspan=2)

        tk.Label(self.search_frame, text="Query").grid(row=1, column=0)
        self.search_entry = tk.Entry(self.search_frame)
        self.search_entry.grid(row=1, column=1)

        tk.Label(self.search_frame, text="Search by").grid(row=2, column=0)
        self.search_by = tk.StringVar(value="title")
        tk.OptionMenu(self.search_frame, self.search_by, "title", "genre").grid(row=2, column=1)

        self.search_button = tk.Button(self.search_frame, text="Search", command=self.search_movies, bg='green', fg='white')
        self.search_button.grid(row=3, column=0, columnspan=2, pady=5)

        # Recommend Movies Frame
        self.recommend_frame = tk.Frame(self.root, bd=2, relief=tk.SUNKEN)
        self.recommend_frame.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")

        tk.Label(self.recommend_frame, text="Recommend Movies").grid(row=0, column=0, columnspan=2)

        tk.Label(self.recommend_frame, text="Top N").grid(row=1, column=0)
        self.top_n_entry = tk.Entry(self.recommend_frame)
        self.top_n_entry.grid(row=1, column=1)

        self.recommend_button = tk.Button(self.recommend_frame, text="Recommend", command=self.recommend_from_search, bg='purple', fg='white')
        self.recommend_button.grid(row=2, column=0, columnspan=2, pady=5)

        # Delete Movie Frame
        self.delete_frame = tk.Frame(self.root, bd=2, relief=tk.SUNKEN)
        self.delete_frame.grid(row=1, column=1, padx=10, pady=10, sticky="nsew")

        tk.Label(self.delete_frame, text="Delete Movie").grid(row=0, column=0, columnspan=2)

        tk.Label(self.delete_frame, text="Title").grid(row=1, column=0)
        self.delete_entry = tk.Entry(self.delete_frame)
        self.delete_entry.grid(row=1, column=1)

        self.delete_button = tk.Button(self.delete_frame, text="Delete", command=self.delete_movie, bg='red', fg='white')
        self.delete_button.grid(row=2, column=0, columnspan=2, pady=5)

        # Result Area
        self.result_frame = tk.Frame(self.root, bd=2, relief=tk.SUNKEN)
        self.result_frame.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

        self.result_frame.grid_rowconfigure(0, weight=1)
        self.result_frame.grid_columnconfigure(0, weight=1)

        self.result_text = tk.Text(self.result_frame, height=10, width=80)
        self.result_text.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

    def add_movie(self):
        title = self.title_entry.get()
        genre = self.genre_entry.get()
        cast_quality = self.cast_entry.get()
        production = self.production_entry.get()
        budget = self.budget_entry.get()
        box_office = self.box_office_entry.get()
        critical_acclaim = float(self.critical_entry.get())
        self.system.add_movie(title, genre, cast_quality, production, budget, box_office, critical_acclaim)
        self.result_text.insert(tk.END, f"Added: {title}\n")
        
        # Clear the entry fields after adding the movie
        self.title_entry.delete(0, tk.END)
        self.genre_entry.delete(0, tk.END)
        self.cast_entry.delete(0, tk.END)
        self.production_entry.delete(0, tk.END)
        self.budget_entry.delete(0, tk.END)
        self.box_office_entry.delete(0, tk.END)
        self.critical_entry.delete(0, tk.END)

    def search_movies(self):
        query = self.search_entry.get()
        search_by = self.search_by.get()
        results = self.system.search_movies(query, search_by)
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, f"Search results for '{query}' by {search_by}:\n")
        if results:
            for movie in results:
                self.result_text.insert(tk.END, f"{movie}\n")
        else:
            self.result_text.insert(tk.END, "No results found.\n")

    def recommend_from_search(self):
        top_n = int(self.top_n_entry.get())
        query = self.search_entry.get()
        search_by = self.search_by.get()
        search_results = self.system.search_movies(query, search_by)
        results = self.system.recommend_movies(search_results, top_n)
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, f"Top {top_n} recommendations from search results for '{query}' by {search_by}:\n")
        for movie in results:
            self.result_text.insert(tk.END, f"{movie}\n")

    def delete_movie(self):
        title = self.delete_entry.get()
        self.system.delete_movie(title)
        self.result_text.insert(tk.END, f"Deleted: {title}\n")   


In [14]:
if __name__ == "__main__":
        csv_file = "tamil_movies.csv"
        system = MovieRecommendationSystem(csv_file)
    
        root = tk.Tk()
        app = MovieApp(root, system)
        root.mainloop()
    
        system.save_movies(csv_file)